# FLamby integration in Fed-BioMed

This notebook showcases some examples of the integration between [FLamby](https://github.com/owkin/FLamby) and Fed-BioMed.

For a thorough understanding, please visit the Tutorials section of our [documentation](../../../getting-started/what-is-fedbiomed/).

This tutorial assumes that you know and understand the basics of Fed-BioMed, that you have already set up the network component, and are familiar with flow of adding data through the node CLI interface.
For an introduction to Fed-BioMed, please follow our PyTorch MNIST [tutorial](../../pytorch/01_PyTorch_MNIST_Single_Node_Tutorial/).

## Downloading FLamby datasets

Before using FLamby, you need to [download the FLamby datasets](https://github.com/owkin/FLamby/blob/main/flamby/datasets/fed_heart_disease/README.md#download-and-preprocessing-instructions) that you plan to use. For licensing reasons, these are not including directly in the FLamby installation.

To download the `fed_ixi` dataset in `${FEDBIOMED_DIR}/data`, follow [FLamby download instructions](https://github.com/owkin/FLamby/blob/main/flamby/datasets/fed_ixi/README.md#download). In a nutshell:
* execute on the **researcher**
```bash
source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment researcher
pip install nibabel
```
* then execute on each **node** (where `${FEDBIOMED_DIR}` is the base directory of Fed-BioMed):
```bash
source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment node
pip install nibabel
python $(find $CONDA_PREFIX -path */fed_ixi/dataset_creation_scripts/download.py) -o ${FEDBIOMED_DIR}/data
```

To download the `fed_heart_disease` dataset in `${FEDBIOMED_DIR}/data`, follow [FLamby download instructions](https://github.com/owkin/FLamby/blob/main/flamby/datasets/fed_heart_disease/README.md#download-and-preprocessing-instructions). In a nutshell:
* execute on the **researcher**
```bash
source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment researcher
pip install wget
```
* then execute on each **node** (where `${FEDBIOMED_DIR}` is the base directory of Fed-BioMed):
```bash
source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment node
pip install wget
python $(find $CONDA_PREFIX -path */fed_heart_disease/dataset_creation_scripts/download.py) --output-folder ${FEDBIOMED_DIR}/data
```


## Install dependencies

If you haven't done so already, install the additional dependencies required by the flamby datasets/features that you intend on using. 

You may check out which dependencies are needed by each dataset directly from Flamby's `setup.py` [file](https://github.com/owkin/FLamby/blob/main/setup.py#L42). In our case we'll be using the federated IXI and federated heart disease datasets, hence we'll need wget, monai and nibabel. 

In [1]:
! pip install wget nibabel  # monai comes already packaged within fed-biomed

  Using cached wget-3.2-py3-none-any.whl
  Using cached nibabel-5.2.1-py3-none-any.whl.metadata (8.8 kB)
Using cached nibabel-5.2.1-py3-none-any.whl (3.3 MB)


## Running a FLamby experiment in a federated setting with Fed-BioMed

Before running a federated experiment, we need to add a FLamby dataset to a node.
From a terminal, `cd` to the Fed-BioMed root installation directory and run

```bash
  ./scripts/fedbiomed_run node dataset add
 ```

Then follow these instructions:
   * Select option 6 (`flamby`) when prompted about the data type
   * type any name for the database (suggested `flamby-ixi`), press <kbd>Enter</kbd> to continue
   * type `flixi` when prompted for tags, press <kbd>Enter</kbd> to continue
   * type any description (suggested `flamby-ixi`), press <kbd>Enter</kbd> to continue
   * select option 3 (`fed_ixi`) when prompted for the FLamby dataset to be configured
   * type a number in the given range, press <kbd>Enter</kbd> to continue
   * type any description for the data loading plan (suggested `flamby-ixi-dlp`), press <kbd>Enter</kbd> to continue


Optionally, repeat the instructions above for the `fed_heart_disease` dataset, using `flheart` for tags.

Finally, start the node with

```bash
  ./scripts/fedbiomed_run node start
 ```


## Basic example: Fed-IXI

The first example will use the model, optimizer and loss function provided by FLamby for the [IXI](https://github.com/owkin/FLamby/tree/main/flamby/datasets/fed_ixi) dataset.

The instructions for using FLamby are:

- define a `TorchTrainingPlan`
- in the `training_data` function, instantiate a `FlambyDataset`
- make sure to include the necessary dependencies in the `init_dependencies` function


In [2]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from flamby.datasets.fed_ixi import Baseline, BaselineLoss, Optimizer
from fedbiomed.common.data import FlambyDataset, DataManager


class MyTrainingPlan(TorchTrainingPlan):
    def init_model(self, model_args):
        return Baseline()

    def init_optimizer(self, optimizer_args):
        return Optimizer(self.model().parameters(), lr=optimizer_args["lr"])

    def init_dependencies(self):
        return ["from flamby.datasets.fed_ixi import Baseline, BaselineLoss, Optimizer",
                "from fedbiomed.common.data import FlambyDataset, DataManager"]

    def training_step(self, data, target):
        output = self.model().forward(data)
        return BaselineLoss().forward(output, target)

    def training_data(self):
        dataset = FlambyDataset()
        loader_arguments = { 'shuffle': True}
        return DataManager(dataset, **loader_arguments)

In [3]:
model_args = {}

training_args = {
    'loader_args': { 'batch_size': 8, },
    'optimizer_args': {
        "lr" : 1e-3
    },
    'epochs': 1,
    'dry_run': False,
    'batch_maxnum': 2 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

In [5]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage
from fedbiomed.researcher.secagg import SecureAggregation

tags =  ['flixi']
rounds = 3

exp_sec_agg_lom = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                secagg=SecureAggregation(active=True, scheme='flamingo'),
                 # or custom SecureAggregation(active=<bool>, clipping_range=<int>)
                  tensorboard=True,
                 node_selection_strategy=None)

2024-05-24 21:13:56,315 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2024-05-24 21:13:56,322 fedbiomed DEBUG - Node: NODE_7849b47e-12fc-4062-a85a-706aa967e395 polling for the tasks

2024-05-24 21:13:56,325 fedbiomed DEBUG - Node: NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 polling for the tasks

2024-05-24 21:13:56,327 fedbiomed DEBUG - Node: NODE_1ab3470a-b961-4de6-a454-237155e67b4c polling for the tasks

2024-05-24 21:13:56,330 fedbiomed INFO - Node selected for training -> NODE_7849b47e-12fc-4062-a85a-706aa967e395

2024-05-24 21:13:56,331 fedbiomed INFO - Node selected for training -> NODE_1ab3470a-b961-4de6-a454-237155e67b4c

2024-05-24 21:13:56,332 fedbiomed INFO - Node selected for training -> NODE_efae9061-c6e3-4c90-87eb-ba8775edd565

2024-05-24 21:13:56,334 fedbiomed DEBUG - Model file has been saved: /workspaces/Projects/fedbiomed/var/experiments/Experiment_0033/model_8bac2645-fa8d-4b4d-af5b-ded985597eb0.py

In [12]:
exp_sec_agg_lom.run()

2024-05-24 21:15:18,592 fedbiomed INFO - Sampled nodes in round 0 ['NODE_7849b47e-12fc-4062-a85a-706aa967e395', 'NODE_1ab3470a-b961-4de6-a454-237155e67b4c', 'NODE_efae9061-c6e3-4c90-87eb-ba8775edd565']

2024-05-24 21:15:18,598 fedbiomed INFO - Sending request 
					 To: NODE_7849b47e-12fc-4062-a85a-706aa967e395 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-24 21:15:18,599 fedbiomed INFO - Sending request 
					 To: NODE_1ab3470a-b961-4de6-a454-237155e67b4c 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-24 21:15:18,601 fedbiomed INFO - Sending request 
					 To: NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-24 21:15:18,643 fedbiomed DEBUG - Node: NODE_7849b47e-12fc-4062-a85a-706aa967e395 polling for the tasks

...

2024-05-24 21:16:48,705 fedbiomed INFO - Aggregating encrypted parameters. This process may take some time depending on model size.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2024-05-24 21:16:48,962 fedbiomed INFO - Aggregating parameters from 3 nodes.

2024-05-24 21:16:49,244 fedbiomed DEBUG - Aggregation is completed in 0.44 seconds.

3

In [15]:
print("\nList the training rounds : ", exp_sec_agg_lom.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
for rnd in range(3):
    round_data = exp_sec_agg_lom.training_replies()[rnd]
    for r in round_data.values():
        print("\t- {id} :\
        \n\t\trtime_training={rtraining:.2f} seconds\
        \n\t\tptime_training={ptraining:.2f} seconds\
        \n\t\ttime_encrypt={time_encrypt:.2f} seconds\
        \n\t\tptxt_model_size={ptxt_model_size:.2f} MB\
        \n\t\tctxt_model_size={ctxt_model_size:.2f} MB\
        \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
            rtraining = r['timing']['rtime_training'],
            ptraining = r['timing']['ptime_training'],
            time_encrypt = r['timing']['time_encrypt'],
            ptxt_model_size = r['communication']['ptxt_model_size'],
            ctxt_model_size = r['communication']['ctxt_model_size'],
            rtotal = r['timing']['rtime_total'],))
    print('\n')


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- NODE_7849b47e-12fc-4062-a85a-706aa967e395 :        
		rtime_training=26.88 seconds        
		ptime_training=70.82 seconds        
		time_encrypt=0.44 seconds        
		ptxt_model_size=15.06 MB        
		ctxt_model_size=16.94 MB        
		rtime_total=27.87 seconds
	- NODE_1ab3470a-b961-4de6-a454-237155e67b4c :        
		rtime_training=26.20 seconds        
		ptime_training=68.51 seconds        
		time_encrypt=0.62 seconds        
		ptxt_model_size=15.06 MB        
		ctxt_model_size=18.35 MB        
		rtime_total=27.87 seconds
	- NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 :        
		rtime_training=26.94 seconds        
		ptime_training=70.61 seconds        
		time_encrypt=0.43 seconds        
		ptxt_model_size=15.06 MB        
		ctxt_model_size=18.35 MB        
		rtime_total=27.87 seconds


	- NODE_7849b47e-12fc-4062-a85a-706aa967e395 :        
		rtime_training=25.17 seco

In [20]:
exp_sec_agg_jls = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                secagg=SecureAggregation(active=True, scheme='jls'),
                 # or custom SecureAggregation(active=<bool>, clipping_range=<int>)
                  tensorboard=True,
                 node_selection_strategy=None)

2024-05-24 21:20:24,361 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2024-05-24 21:20:24,367 fedbiomed DEBUG - Node: NODE_7849b47e-12fc-4062-a85a-706aa967e395 polling for the tasks

2024-05-24 21:20:24,370 fedbiomed DEBUG - Node: NODE_1ab3470a-b961-4de6-a454-237155e67b4c polling for the tasks

2024-05-24 21:20:24,372 fedbiomed DEBUG - Node: NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 polling for the tasks

2024-05-24 21:20:24,376 fedbiomed INFO - Node selected for training -> NODE_7849b47e-12fc-4062-a85a-706aa967e395

2024-05-24 21:20:24,377 fedbiomed INFO - Node selected for training -> NODE_1ab3470a-b961-4de6-a454-237155e67b4c

2024-05-24 21:20:24,377 fedbiomed INFO - Node selected for training -> NODE_efae9061-c6e3-4c90-87eb-ba8775edd565

2024-05-24 21:20:24,380 fedbiomed DEBUG - Model file has been saved: /workspaces/Projects/fedbiomed/var/experiments/Experiment_0036/model_77d992e8-479a-4bca-b052-b5abd55417ae.py

In [21]:
exp_sec_agg_jls.run()

2024-05-24 21:20:25,203 fedbiomed DEBUG - Secagg context for default_biprime0 is already existing on researcher researcher_id='RESEARCHER_596f170d-96d8-417a-8ddf-9bdb1cf7d52b'

2024-05-24 21:20:25,209 fedbiomed DEBUG - Node: NODE_1ab3470a-b961-4de6-a454-237155e67b4c polling for the tasks

2024-05-24 21:20:25,211 fedbiomed DEBUG - Node: NODE_7849b47e-12fc-4062-a85a-706aa967e395 polling for the tasks

2024-05-24 21:20:25,213 fedbiomed DEBUG - Node: NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 polling for the tasks

2024-05-24 21:20:25,225 fedbiomed DEBUG - Node: NODE_7849b47e-12fc-4062-a85a-706aa967e395 polling for the tasks

...

2024-05-24 21:26:53,777 fedbiomed INFO - Nodes that successfully reply in round 2 ['NODE_7849b47e-12fc-4062-a85a-706aa967e395', 'NODE_1ab3470a-b961-4de6-a454-237155e67b4c', 'NODE_efae9061-c6e3-4c90-87eb-ba8775edd565']

2024-05-24 21:26:53,794 fedbiomed INFO - Aggregating encrypted parameters. This process may take some time depending on model size.

2024-05-24 21:27:45,332 fedbiomed INFO - Aggregating 3 parameters from 3 nodes.

2024-05-24 21:27:45,526 fedbiomed DEBUG - Aggregation is completed in 51.2 seconds.

3

In [23]:
print("\nList the training rounds : ", exp_sec_agg_jls.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
for rnd in range(rounds):
    round_data = exp_sec_agg_jls.training_replies()[rnd]
    for r in round_data.values():
        print("\t- {id} :\
        \n\t\trtime_training={rtraining:.2f} seconds\
        \n\t\tptime_training={ptraining:.2f} seconds\
        \n\t\ttime_encrypt={time_encrypt:.2f} seconds\
        \n\t\tptxt_model_size={ptxt_model_size:.2f} MB\
        \n\t\tctxt_model_size={ctxt_model_size:.2f} MB\
        \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
            rtraining = r['timing']['rtime_training'],
            ptraining = r['timing']['ptime_training'],
            time_encrypt = r['timing']['time_encrypt'],
            ptxt_model_size = r['communication']['ptxt_model_size'],
            ctxt_model_size = r['communication']['ctxt_model_size'],
            rtotal = r['timing']['rtime_total'],))
    print('\n')


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- NODE_7849b47e-12fc-4062-a85a-706aa967e395 :        
		rtime_training=25.29 seconds        
		ptime_training=67.10 seconds        
		time_encrypt=49.37 seconds        
		ptxt_model_size=15.06 MB        
		ctxt_model_size=4.84 MB        
		rtime_total=76.57 seconds
	- NODE_1ab3470a-b961-4de6-a454-237155e67b4c :        
		rtime_training=25.93 seconds        
		ptime_training=68.13 seconds        
		time_encrypt=49.13 seconds        
		ptxt_model_size=15.06 MB        
		ctxt_model_size=4.84 MB        
		rtime_total=76.57 seconds
	- NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 :        
		rtime_training=27.37 seconds        
		ptime_training=72.88 seconds        
		time_encrypt=48.80 seconds        
		ptxt_model_size=15.06 MB        
		ctxt_model_size=4.84 MB        
		rtime_total=76.57 seconds


	- NODE_7849b47e-12fc-4062-a85a-706aa967e395 :        
		rtime_training=30.06 seco

Save trained model to file

In [ ]:
exp.training_plan().export_model('./trained_model')

## Basic example: Fed-Heart-Disease

We showcase similar functionalities as the above fed-ixi case, but with FLamby's [Heart Disease](https://github.com/owkin/FLamby/tree/main/flamby/datasets/fed_heart_disease) dataset.

In [ ]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from flamby.datasets.fed_heart_disease import Baseline, BaselineLoss, Optimizer
from fedbiomed.common.data import FlambyDataset, DataManager

class FedHeartTrainingPlan(TorchTrainingPlan):
    def init_model(self, model_args):
        return Baseline()

    def init_optimizer(self, optimizer_args):
        return Optimizer(self.model().parameters(), lr=optimizer_args["lr"])

    def init_dependencies(self):
        return ["from flamby.datasets.fed_heart_disease import Baseline, BaselineLoss, Optimizer",
                "from fedbiomed.common.data import FlambyDataset, DataManager"]

    def training_step(self, data, target):
        output = self.model().forward(data)
        return BaselineLoss().forward(output, target)

    def training_data(self):
        dataset = FlambyDataset()
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset, **train_kwargs)

In [ ]:
training_args = {
    'loader_args': { 'batch_size': 4, },
    'optimizer_args': {
        'lr': 0.001,
    },
    'epochs': 1,
    'dry_run': False,
    'log_interval': 2,
    'batch_maxnum': 8,
    'test_ratio' : 0.0,
    'test_on_global_updates': False,
    'test_on_local_updates': False,
}

model_args = {}

In [ ]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['flheart']
num_rounds = 1

exp = Experiment(tags=tags,
                 training_plan_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 model_args=model_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp.run_once(increase=True)

## Complex example: Fed-IXI with data preprocessing and custom training elements

This example demonstrates how to define transformations for data preprocessing and provide a customized model, optimizer, and loss function.
Incidentally, it also shows how to use `model_args` and `training_args` to parametrize the model, optimizer, and training loop.

### Definition of preprocessing transforms

This is achieved in the `training_data` function.
After instantiating the `FlambyDataset`, you may use the `init_transform` function to attach a preprocessing transformation for your data.
Note that the transform that you define must be of type `torchvision.transforms.Compose` or `monai.transforms.Compose`.

### Definition of custom model, optimizer and loss

This is achieved just like any `TorchTrainingPlan`, through the functions `init_model`, `init_optimizer`, and `training_step`.

In [ ]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import AdamW
from torch import nn
import torch.nn.functional as F
from unet import UNet
from monai.transforms import Compose, NormalizeIntensity, Resize

class UNetTrainingPlan(TorchTrainingPlan):

    class MyUNet(nn.Module):
        CHANNELS_DIMENSION = 1

        def __init__(self, model_args):
            super().__init__()
            self.unet = UNet(
            in_channels = model_args.get('in_channels',1),
            out_classes = model_args.get('out_classes',2),
            dimensions = model_args.get('dimensions',2),
            num_encoding_blocks = model_args.get('num_encoding_blocks',5),
            out_channels_first_layer = model_args.get('out_channels_first_layer',64),
            normalization = model_args.get('normalization', None),
            pooling_type = model_args.get('pooling_type', 'max'),
            upsampling_type = model_args.get('upsampling_type','conv'),
            preactivation = model_args.get('preactivation',False),
            residual = model_args.get('residual',False),
            padding = model_args.get('padding',0),
            padding_mode = model_args.get('padding_mode','zeros'),
            activation = model_args.get('activation','ReLU'),
            initial_dilation = model_args.get('initial_dilation',None),
            dropout = model_args.get('dropout',0),
            monte_carlo_dropout = model_args.get('monte_carlo_dropout',0)
        )

        def forward(self, x):
            x = self.unet.forward(x)
            x = F.softmax(x, dim=UNetTrainingPlan.MyUNet.CHANNELS_DIMENSION)
            return x

    def init_model(self, model_args):
        return UNetTrainingPlan.MyUNet(model_args)

    def init_dependencies(self):
        return ["from torch import nn",
               'import torch.nn.functional as F',
               'from torch.optim import AdamW',
               'from unet import UNet',
               'from monai.transforms import Compose, NormalizeIntensity, Resize',
                'from fedbiomed.common.data import FlambyDataset']

    def init_optimizer(self, optimizer_args):
        return AdamW(self.model().parameters(),
                     lr=optimizer_args["lr"],
                     betas=optimizer_args["betas"],
                     eps=optimizer_args["eps"])

    @staticmethod
    def get_dice_loss(output, target, epsilon=1e-9):
        SPATIAL_DIMENSIONS = 2, 3, 4
        p0 = output
        g0 = target
        p1 = 1 - p0
        g1 = 1 - g0
        tp = (p0 * g0).sum(dim=SPATIAL_DIMENSIONS)
        fp = (p0 * g1).sum(dim=SPATIAL_DIMENSIONS)
        fn = (p1 * g0).sum(dim=SPATIAL_DIMENSIONS)
        num = 2 * tp
        denom = 2 * tp + fp + fn + epsilon
        dice_score = num / denom
        return 1. - dice_score

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss = UNetTrainingPlan.get_dice_loss(output, target)
        avg_loss = loss.mean()
        return avg_loss

    def testing_step(self, data, target):
        prediction = self.model().forward(data)
        loss = UNetTrainingPlan.get_dice_loss(prediction, target)
        avg_loss = loss.mean()  # average per batch
        return avg_loss

    def training_data(self):
        dataset = FlambyDataset()
        transform = Compose([Resize((48,60,48)), NormalizeIntensity()])
        dataset.init_transform(transform)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset, **train_kwargs)

In [ ]:
model_args = {
    'in_channels': 1,
    'out_classes': 2,
    'dimensions': 3,
    'num_encoding_blocks': 3,
    'out_channels_first_layer': 8,
    'normalization': 'batch',
    'upsampling_type': 'linear',
    'padding': True,
    'activation': 'PReLU',
}

training_args = {
    'loader_args': { 'batch_size': 16, },
    'optimizer_args': {
        'lr': 0.001,
        'betas': (0.9, 0.999),
        'eps': 1e-08
    },
    'epochs': 1,
    'dry_run': False,
    'log_interval': 2,
    'test_ratio' : 0.0,
    'test_on_global_updates': False,
    'test_on_local_updates': False,
    'batch_maxnum': 2 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}



In [ ]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['flixi']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=UNetTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp.run_once(increase=True)